In [1]:
import copy
import glob
import os
import time

import gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from utils import orthogonal

from all_stuff import *

from envs import make_env # had to manually add some files into directory for env to reference bc baselines 
# modules not working right

#from storage import RolloutStorage

In [9]:
class args:
    def __init__(self):
        self.env_name='PongNoFrameskip-v4'
        self.seed=1
        self.log_dir=''
        self.save_dir='saved_models'
        self.cuda=False
        self.algo='a2c'
        self.num_stack=4
        self.num_steps=5
        self.num_processes=2
        self.lr=7e-4
        self.eps=1e-5
        self.alpha=.99
        self.max_grad_norm=.5
        self.value_loss_coef=.5
        self.entropy_coef=.1
        self.num_frames=8e6
        self.use_gae=False
        self.gamma=.99
        self.tau=.95
        self.save_interval=1000
        self.log_interval=100
        self.from_saved=False
        
args = args()

SAVE_PATH = "saved_models/pong_112917.pt"

num_updates = int(args.num_frames) // args.num_steps // args.num_processes

torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

In [36]:

os.environ['OMP_NUM_THREADS'] = '1'

envs = [make_env(args.env_name, args.seed, i, args.log_dir) for i in range(args.num_processes)]

if args.num_processes > 1:
    envs = SubprocVecEnv(envs)
else:
    envs = DummyVecEnv(envs)

if len(envs.observation_space.shape) == 1:
    envs = VecNormalize(envs)

obs_shape = envs.observation_space.shape
obs_shape = (obs_shape[0] * args.num_stack, *obs_shape[1:]) #4 x 84 x 84

actor_critic = CNNPolicy(obs_shape[0], envs.action_space)

if args.from_saved:
    print("loading saved model from " + SAVE_PATH)
    actor_critic.load_state_dict(torch.load(SAVE_PATH))

if envs.action_space.__class__.__name__ == "Discrete":
    action_shape = 1
else:
    action_shape = envs.action_space.shape[0]

if args.cuda:
    actor_critic.cuda()

optimizer = optim.RMSprop(actor_critic.parameters(), args.lr, eps=args.eps, alpha=args.alpha)

rollouts = RolloutStorage(args.num_steps, args.num_processes, obs_shape, envs.action_space)
current_obs = torch.zeros(args.num_processes, *obs_shape)

def update_current_obs(obs):
    shape_dim0 = envs.observation_space.shape[0]
    obs = torch.from_numpy(obs).float()
    if args.num_stack > 1:
        current_obs[:, :-shape_dim0] = current_obs[:, shape_dim0:]
    current_obs[:, -shape_dim0:] = obs

obs = envs.reset()
update_current_obs(obs)

rollouts.observations[0].copy_(current_obs)

# These variables are used to compute average rewards for all processes.
episode_rewards = torch.zeros([args.num_processes, 1])
final_rewards = torch.zeros([args.num_processes, 1])

if args.cuda:
    current_obs = current_obs.cuda()
    rollouts.cuda()

start = time.time()
for j in range(num_updates):
    for step in range(args.num_steps):
        # Sample actions
        value, action, action_log_prob, dist_entropy = actor_critic(Variable(rollouts.observations[step], volatile=True),
                                                                  Variable(rollouts.masks[step], volatile=True))
        
        print("\n Stepping \nvalue", value, "\naction", action, "\naction_log_prob", action_log_prob,\
              "\ndist_entropy",dist_entropy)
        cpu_actions = action.data.squeeze(1).cpu().numpy()

        # Obser reward and next obs
        obs, reward, done, info = envs.step(cpu_actions)
        reward = torch.from_numpy(np.expand_dims(np.stack(reward), 1)).float()
        episode_rewards += reward

        # If done then clean the history of observations.
        masks = torch.FloatTensor([[0.0] if done_ else [1.0] for done_ in done])
        final_rewards *= masks
        final_rewards += (1 - masks) * episode_rewards
        episode_rewards *= masks

        if args.cuda:
            masks = masks.cuda()

        if current_obs.dim() == 4:
            current_obs *= masks.unsqueeze(2).unsqueeze(2)
        else:
            current_obs *= masks

        update_current_obs(obs)
        rollouts.insert(step, current_obs, action.data, action_log_prob.data, value.data, reward, masks)
    
    print("################################\nDone stepping")

    next_value = actor_critic(Variable(rollouts.observations[-1], volatile=True),
                              Variable(rollouts.masks[-1], volatile=True))[0].data

    rollouts.compute_returns(next_value, args.use_gae, args.gamma, args.tau)

    
    values, actions, action_log_probs, dist_entropy = actor_critic(Variable(rollouts.observations[:-1].view(-1, *obs_shape)),
                                                           Variable(rollouts.masks[:-1].view(-1, 1)))
    #Variable(rollouts.actions.view(-1, action_shape))
    
    values = values.view(args.num_steps, args.num_processes, 1)
    action_log_probs = action_log_probs.view(args.num_steps, args.num_processes, 1)
    
    # this line gets out of "diff stoch variables req reward" error
    # action_log_probs = Variable(action_log_probs.data, requires_grad=False)

    advantages = Variable(rollouts.returns[:-1]) - values
    value_loss = advantages.pow(2).mean()

    #action_log_probs is the one causing the error
    action_loss = -(Variable(advantages.data) * action_log_probs).mean()
    print("\nnext value", next_value, "\nvalues", values, "\nactions", actions,\
          "\naction_log_probs", action_log_probs, "\nreturns", rollouts.returns[:-1],
          "\ndist_entropy", dist_entropy, "\nadvantages", advantages)

    optimizer.zero_grad()
    
    (value_loss * args.value_loss_coef + action_loss - dist_entropy * args.entropy_coef).backward()

    nn.utils.clip_grad_norm(actor_critic.parameters(), args.max_grad_norm)

    optimizer.step()

    rollouts.after_update()

    if j % args.save_interval == 0 and args.save_dir != "":

        # A really ugly way to save a model to CPU
        save_model = actor_critic
        if args.cuda:
            save_model = copy.deepcopy(actor_critic).cpu() # save THIS one

        torch.save(save_model.state_dict(), SAVE_PATH)

    if j % args.log_interval == 0:
        end = time.time()
        total_num_steps = (j + 1) * args.num_processes * args.num_steps
        print("Updates {}, num timesteps {}, FPS {}, mean/median reward {:.1f}/{:.1f}, min/max reward {:.1f}/{:.1f}, entropy {:.5f}, value loss {:.5f}, policy loss {:.5f}".
            format(j, total_num_steps,
                   int(total_num_steps / (end - start)),
                   final_rewards.mean(),
                   final_rewards.median(),
                   final_rewards.min(),
                   final_rewards.max(), dist_entropy.data[0],
                   value_loss.data[0], action_loss.data[0]))


 Stepping 
value Variable containing:
1.00000e-02 *
 -8.2926
 -8.0802
[torch.FloatTensor of size 2x1]
 
action Variable containing:
 2
 1
[torch.LongTensor of size 2x1]
 
action_log_prob Variable containing:
-1.9392
-1.3881
[torch.FloatTensor of size 2x1]
 
dist_entropy Variable containing:
 1.7634
[torch.FloatTensor of size 1]


 Stepping 
value Variable containing:
1.00000e-02 *
 -5.2210
 -6.2289
[torch.FloatTensor of size 2x1]
 
action Variable containing:
 0
 3
[torch.LongTensor of size 2x1]
 
action_log_prob Variable containing:
-1.8058
-1.7931
[torch.FloatTensor of size 2x1]
 
dist_entropy Variable containing:
 1.7549
[torch.FloatTensor of size 1]


 Stepping 
value Variable containing:
1.00000e-02 *
 -5.2485
 -5.6389
[torch.FloatTensor of size 2x1]
 
action Variable containing:
 3
 3
[torch.LongTensor of size 2x1]
 
action_log_prob Variable containing:
-1.6338
-1.6338
[torch.FloatTensor of size 2x1]
 
dist_entropy Variable containing:
 1.7407
[torch.FloatTensor of size 1]


 St

 
returns 
(0 ,.,.) = 
  0.3695
  0.3715

(1 ,.,.) = 
  0.3733
  0.3752

(2 ,.,.) = 
  0.3770
  0.3790

(3 ,.,.) = 
  0.3808
  0.3829

(4 ,.,.) = 
  0.3847
  0.3867
[torch.FloatTensor of size 5x2x1]
 
dist_entropy Variable containing:
 1.7827
[torch.FloatTensor of size 1]
 
advantages Variable containing:
(0 ,.,.) = 
1.00000e-02 *
  -1.9728
  -1.8750

(1 ,.,.) = 
1.00000e-02 *
  -1.7008
  -1.2465

(2 ,.,.) = 
1.00000e-02 *
  -1.5318
  -0.7725

(3 ,.,.) = 
1.00000e-02 *
  -1.0190
  -1.1809

(4 ,.,.) = 
1.00000e-02 *
  -0.4385
  -0.2199
[torch.FloatTensor of size 5x2x1]


 Stepping 
value Variable containing:
1.00000e-02 *
 -8.1458
 -8.2304
[torch.FloatTensor of size 2x1]
 
action Variable containing:
 0
 3
[torch.LongTensor of size 2x1]
 
action_log_prob Variable containing:
-1.7798
-1.7923
[torch.FloatTensor of size 2x1]
 
dist_entropy Variable containing:
 1.7909
[torch.FloatTensor of size 1]


 Stepping 
value Variable containing:
1.00000e-02 *
 -8.1218
 -7.7435
[torch.FloatTensor of


 Stepping 
value Variable containing:
1.00000e-02 *
 -1.5834
 -2.1126
[torch.FloatTensor of size 2x1]
 
action Variable containing:
 1
 5
[torch.LongTensor of size 2x1]
 
action_log_prob Variable containing:
-1.8015
-1.7830
[torch.FloatTensor of size 2x1]
 
dist_entropy Variable containing:
 1.7916
[torch.FloatTensor of size 1]


 Stepping 
value Variable containing:
1.00000e-02 *
 -1.7552
 -1.5500
[torch.FloatTensor of size 2x1]
 
action Variable containing:
 5
 1
[torch.LongTensor of size 2x1]
 
action_log_prob Variable containing:
-1.7832
-1.8003
[torch.FloatTensor of size 2x1]
 
dist_entropy Variable containing:
 1.7916
[torch.FloatTensor of size 1]


 Stepping 
value Variable containing:
1.00000e-02 *
 -1.8053
 -1.4120
[torch.FloatTensor of size 2x1]
 
action Variable containing:
 4
 4
[torch.LongTensor of size 2x1]
 
action_log_prob Variable containing:
-1.8176
-1.8182
[torch.FloatTensor of size 2x1]
 
dist_entropy Variable containing:
 1.7916
[torch.FloatTensor of size 1]


 St


 Stepping 
value Variable containing:
-0.5052
-0.5026
[torch.FloatTensor of size 2x1]
 
action Variable containing:
 4
 3
[torch.LongTensor of size 2x1]
 
action_log_prob Variable containing:
-1.9052
-1.8508
[torch.FloatTensor of size 2x1]
 
dist_entropy Variable containing:
 1.7845
[torch.FloatTensor of size 1]


 Stepping 
value Variable containing:
-0.4993
-0.5026
[torch.FloatTensor of size 2x1]
 
action Variable containing:
 0
 3
[torch.LongTensor of size 2x1]
 
action_log_prob Variable containing:
-1.9017
-1.8548
[torch.FloatTensor of size 2x1]
 
dist_entropy Variable containing:
 1.7845
[torch.FloatTensor of size 1]


 Stepping 
value Variable containing:
-0.5023
-0.5042
[torch.FloatTensor of size 2x1]
 
action Variable containing:
 5
 2
[torch.LongTensor of size 2x1]
 
action_log_prob Variable containing:
-1.8676
-1.6320
[torch.FloatTensor of size 2x1]
 
dist_entropy Variable containing:
 1.7846
[torch.FloatTensor of size 1]


 Stepping 
value Variable containing:
-0.5035
-0.50


 Stepping 
value Variable containing:
1.00000e-02 *
 -2.2998
 -1.9220
[torch.FloatTensor of size 2x1]
 
action Variable containing:
 3
 5
[torch.LongTensor of size 2x1]
 
action_log_prob Variable containing:
-1.7867
-1.7945
[torch.FloatTensor of size 2x1]
 
dist_entropy Variable containing:
 1.7917
[torch.FloatTensor of size 1]


 Stepping 
value Variable containing:
1.00000e-02 *
 -1.7172
 -1.8729
[torch.FloatTensor of size 2x1]
 
action Variable containing:
 0
 4
[torch.LongTensor of size 2x1]
 
action_log_prob Variable containing:
-1.8099
-1.7702
[torch.FloatTensor of size 2x1]
 
dist_entropy Variable containing:
 1.7917
[torch.FloatTensor of size 1]


 Stepping 
value Variable containing:
1.00000e-02 *
 -2.0462
 -2.1888
[torch.FloatTensor of size 2x1]
 
action Variable containing:
 4
 5
[torch.LongTensor of size 2x1]
 
action_log_prob Variable containing:
-1.7691
-1.7968
[torch.FloatTensor of size 2x1]
 
dist_entropy Variable containing:
 1.7917
[torch.FloatTensor of size 1]


 St

...


 Stepping 
value Variable containing:
-0.6614
-0.6623
[torch.FloatTensor of size 2x1]
 
action Variable containing:
 5
 4
[torch.LongTensor of size 2x1]
 
action_log_prob Variable containing:
-1.6870
-1.8768
[torch.FloatTensor of size 2x1]
 
dist_entropy Variable containing:
 1.7883
[torch.FloatTensor of size 1]


 Stepping 
value Variable containing:
-0.6627
-0.6614
[torch.FloatTensor of size 2x1]
 
action Variable containing:
 2
 5
[torch.LongTensor of size 2x1]
 
action_log_prob Variable containing:
-1.7242
-1.6879
[torch.FloatTensor of size 2x1]
 
dist_entropy Variable containing:
 1.7883
[torch.FloatTensor of size 1]


 Stepping 
value Variable containing:
-0.6630
-0.6593
[torch.FloatTensor of size 2x1]
 
action Variable containing:
 2
 5
[torch.LongTensor of size 2x1]
 
action_log_prob Variable containing:
-1.7252
-1.6871
[torch.FloatTensor of size 2x1]
 
dist_entropy Variable containing:
 1.7883
[torch.FloatTensor of size 1]


 Stepping 
value Variable containing:
-0.6633
-0.66

Process Process-69:
Process Process-70:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):



 Stepping 
value Variable containing:
-0.3639
-0.3603
[torch.FloatTensor of size 2x1]
 
action Variable containing:
 2
 3
[torch.LongTensor of size 2x1]
 
action_log_prob Variable containing:
-1.7541
-1.7671
[torch.FloatTensor of size 2x1]
 
dist_entropy Variable containing:
 1.7908
[torch.FloatTensor of size 1]


 Stepping 
value Variable containing:
-0.3637
-0.3611
[torch.FloatTensor of size 2x1]
 
action Variable containing:
 1
 3
[torch.LongTensor of size 2x1]
 
action_log_prob Variable containing:
-1.8529
-1.7670
[torch.FloatTensor of size 2x1]
 
dist_entropy Variable containing:
 1.7908
[torch.FloatTensor of size 1]


 Stepping 
value Variable containing:
-0.3639
-0.3611
[torch.FloatTensor of size 2x1]
 
action Variable containing:
 1
 5
[torch.LongTensor of size 2x1]
 
action_log_prob Variable containing:
-1.8526
-1.7409
[torch.FloatTensor of size 2x1]
 
dist_entropy Variable containing:
 1.7908
[torch.FloatTensor of size 1]


 Stepping 
value Variable containing:
-0.3638
-0.36

  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/beans/pytorch-a2c-ppo-acktr/all_stuff.py", line 8, in worker
    cmd, data = remote.recv()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/beans/pytorch-a2c-ppo-acktr/all_stuff.py", line 8, in worker
    cmd, data = remote.recv()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
  File 

KeyboardInterrupt: 

In [35]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1 or classname.find('Linear') != -1:
        orthogonal(m.weight.data)
        if m.bias is not None:
            m.bias.data.fill_(0)
    
    
class CNNPolicy(nn.Module):
    def __init__(self, num_inputs, action_space):
        super(CNNPolicy, self).__init__()
        self.conv1 = nn.Conv2d(num_inputs, 32, 8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, 4, stride=2)
        self.conv3 = nn.Conv2d(64, 32, 3, stride=1)

        self.linear1 = nn.Linear(32 * 7 * 7, 512)

        self.critic_linear = nn.Linear(512, 1)

        self.actor_linear = nn.Linear(512, action_space.n)

        self.train()
        self.reset_parameters()

    @property
    def state_size(self):
        return 1

    def forward(self, inputs, masks):
        x = self.conv1(inputs / 255.0)
        x = F.relu(x)

        x = self.conv2(x)
        x = F.relu(x)

        x = self.conv3(x)
        x = F.relu(x)

        x = x.view(-1, 32 * 7 * 7)
        x = self.linear1(x)
        
        x = F.relu(x)
        
        value = self.critic_linear(x)
        
        actions_dist = self.actor_linear(x)
        
        
        # calculates the probabilities of each action available
        probs = F.softmax(actions_dist)
        log_probs = F.log_softmax(actions_dist)
        
        # chooses randomly based on probabilities
        # never defauts to deterministic max value
        action = probs.multinomial() 
        
        # don't want to allow model to change action when differentiating
        action = Variable(action.data, requires_grad=False)
        
        # selects the probability for the action taken
        action_log_prob = log_probs.gather(1, action)
        
        dist_entropy = -(log_probs * probs).sum(-1).mean()
        
        return value, action, action_log_prob, dist_entropy
    
    
    def act(self, inputs, masks, deterministic=False):
        value, x = self(inputs, masks)
        action = self.action_dist_linear.sample(x, deterministic=deterministic)
        action_log_probs, dist_entropy = self.action_dist_linear.logprobs_and_entropy(x, action)
        return value, action, action_log_probs

    def evaluate_actions(self, inputs, masks, actions):
        value, x = self(inputs, masks)
        action_log_probs, dist_entropy = self.action_dist_linear.logprobs_and_entropy(x, actions)
        return value, action_log_probs, dist_entropy
    
    def reset_parameters(self):
        self.apply(weights_init)

        relu_gain = nn.init.calculate_gain('relu')
        self.conv1.weight.data.mul_(relu_gain)
        self.conv2.weight.data.mul_(relu_gain)
        self.conv3.weight.data.mul_(relu_gain)
        self.linear1.weight.data.mul_(relu_gain)
        
        
        


In [3]:
class RolloutStorage(object):
    def __init__(self, num_steps, num_processes, obs_shape, action_space):
        self.observations = torch.zeros(num_steps + 1, num_processes, *obs_shape)
        self.rewards = torch.zeros(num_steps, num_processes, 1)
        self.value_preds = torch.zeros(num_steps + 1, num_processes, 1)
        self.returns = torch.zeros(num_steps + 1, num_processes, 1)
        self.action_log_probs = torch.zeros(num_steps, num_processes, 1)
        if action_space.__class__.__name__ == 'Discrete':
            action_shape = 1
        else:
            action_shape = action_space.shape[0]
        self.actions = torch.zeros(num_steps, num_processes, action_shape)
        if action_space.__class__.__name__ == 'Discrete':
            self.actions = self.actions.long()
        self.masks = torch.ones(num_steps + 1, num_processes, 1)

    def cuda(self):
        self.observations = self.observations.cuda()
        self.rewards = self.rewards.cuda()
        self.value_preds = self.value_preds.cuda()
        self.returns = self.returns.cuda()
        self.action_log_probs = self.action_log_probs.cuda()
        self.actions = self.actions.cuda()
        self.masks = self.masks.cuda()

    def insert(self, step, current_obs, action, action_log_prob, value_pred, reward, mask):
        self.observations[step + 1].copy_(current_obs)
        self.actions[step].copy_(action)
        self.action_log_probs[step].copy_(action_log_prob)
        self.value_preds[step].copy_(value_pred)
        self.rewards[step].copy_(reward)
        self.masks[step + 1].copy_(mask)

    def after_update(self):
        self.observations[0].copy_(self.observations[-1])
        self.masks[0].copy_(self.masks[-1])

    def compute_returns(self, next_value, use_gae, gamma, tau):
        if use_gae:
            self.value_preds[-1] = next_value
            gae = 0
            for step in reversed(range(self.rewards.size(0))):
                delta = self.rewards[step] + gamma * self.value_preds[step + 1] * self.masks[step + 1] - self.value_preds[step]
                gae = delta + gamma * tau * self.masks[step + 1] * gae
                self.returns[step] = gae + self.value_preds[step]
        else:
            self.returns[-1] = next_value
            for step in reversed(range(self.rewards.size(0))):
                self.returns[step] = self.returns[step + 1] * \
                    gamma * self.masks[step + 1] + self.rewards[step]
